In [ ]:
        # "org.apache.spark:spark-avro_2.13:3.4.1"

In [10]:
from pyspark.sql import SparkSession

# -------------------
# Spark session
# -------------------
spark = (
    SparkSession.builder
        .appName("Delta S3 Hive Auto Insert")
        .master("local[*]")
        .config("spark.jars.packages",
                "io.delta:delta-spark_2.13:4.0.0,"
                "org.apache.hadoop:hadoop-aws:3.4.1,"
                "org.apache.spark:spark-avro_2.13:3.4.1")  # <- add this
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        # MinIO / S3
        .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
        .config("spark.hadoop.fs.s3a.access.key", "admin")
        .config("spark.hadoop.fs.s3a.secret.key", "password")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        .config("spark.sql.warehouse.dir", "s3a://warehouse/")
        # Hive Metastore
        .config("spark.hadoop.hive.metastore.uris", "thrift://localhost:9083")
        .config("spark.sql.catalogImplementation", "hive")
        .getOrCreate()
)

print("Spark session started")

# Check current catalog
current_catalog = spark.catalog.currentCatalog()
print("Current catalog:", current_catalog)

# Check current database
current_db = spark.catalog.currentDatabase()
print("Current database:", current_db)

Spark session started
Current catalog: spark_catalog
Current database: default


In [7]:
# Show detailed info including location
spark.sql("DESCRIBE DETAIL silver.silver_ecg").show(truncate=False)

spark.sql("DESCRIBE silver.silver_ecg").show(truncate=False)

# Show first 20 rows
spark.sql("SELECT * FROM silver.silver_ecg").show()

# Count rows
spark.sql("SELECT COUNT(*) FROM silver.silver_ecg").show()

+------+------------------------------------+-------------------------------+-----------+------------------------------------+-----------------------+-------------------+----------------+-----------------+--------+-----------+----------+----------------+----------------+------------------------+
|format|id                                  |name                           |description|location                            |createdAt              |lastModified       |partitionColumns|clusteringColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|tableFeatures           |
+------+------------------------------------+-------------------------------+-----------+------------------------------------+-----------------------+-------------------+----------------+-----------------+--------+-----------+----------+----------------+----------------+------------------------+
|delta |a5f04c28-015e-468a-bbb6-c6d558e5398f|spark_catalog.silver.silver_ecg|NULL       |s3a://warehouse/silv

In [13]:
from pyspark.sql.functions import current_timestamp, col, input_file_name, lit
import boto3
from urllib.parse import urlparse
from pyspark.sql.types import StringType

# 1️⃣ Read Avro
avro_path = "s3a://warehouse/fs/ecg_avro/D_01/*.avro"
df = spark.read.format("avro").load(avro_path)

# 2️⃣ Add the file path column
df = df.withColumn("file_path", input_file_name())

# 3️⃣ Get last modified time from MinIO/S3
s3 = boto3.client(
    "s3",
    endpoint_url="http://localhost:9000",
    aws_access_key_id="admin",
    aws_secret_access_key="password"
)

# Collect distinct file paths
file_paths = [row.file_path for row in df.select("file_path").distinct().collect()]

# Map each file path to its last modified timestamp
file_ts_map = {}
for path in file_paths:
    parsed = urlparse(path)
    bucket = parsed.netloc
    key = parsed.path.lstrip("/")
    file_ts_map[path] = s3.head_object(Bucket=bucket, Key=key)["LastModified"].isoformat()

# 4️⃣ Broadcast the mapping and create a UDF
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def get_file_ts(path):
    return file_ts_map.get(path, None)

get_file_ts_udf = udf(get_file_ts, StringType())

# 5️⃣ Flatten and add timestamps
df_flat = df.select(
    "device_id",
    "record_id",
    "start_time",
    "ts",
    col("signals.ecg_i").alias("ecg_i"),
    col("signals.ecg_i_filtered").alias("ecg_i_filtered"),
    current_timestamp().alias("ingest_ts"),
    get_file_ts_udf(col("file_path")).alias("file_modified_ts")
)

# 6️⃣ Write to Delta table
df_flat.write.format("delta").mode("append").saveAsTable("silver.silver_ecg")

25/12/28 22:27:13 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://warehouse/fs/ecg_avro/D_01/*.avro.
java.io.FileNotFoundException: No such file or directory: s3a://warehouse/fs/ecg_avro/D_01/*.avro
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:4156)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:4007)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$getFileStatus$22(S3AFileSystem.java:3984)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.invokeTrackingDuration(IOStatisticsBinding.java:547)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.lambda$trackDurationOfOperation$5(IOStatisticsBinding.java:528)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.trackDuration(IOStatisticsBinding.java:449)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.trackDurationAndSpan(S3AFileSystem.java:2865)
	at org.apache.hadoop.fs.s3a.S3AFileSyst

In [14]:
# Show detailed info including location
spark.sql("DESCRIBE DETAIL silver.silver_ecg").show(truncate=False)

spark.sql("DESCRIBE silver.silver_ecg").show(truncate=False)

# Show first 20 rows
spark.sql("SELECT * FROM silver.silver_ecg").show()

# Count rows
spark.sql("SELECT COUNT(*) FROM silver.silver_ecg").show()

+------+------------------------------------+-------------------------------+-----------+------------------------------------+-----------------------+-------------------+----------------+-----------------+--------+-----------+----------+----------------+----------------+------------------------+
|format|id                                  |name                           |description|location                            |createdAt              |lastModified       |partitionColumns|clusteringColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|tableFeatures           |
+------+------------------------------------+-------------------------------+-----------+------------------------------------+-----------------------+-------------------+----------------+-----------------+--------+-----------+----------+----------------+----------------+------------------------+
|delta |f9d42291-6e37-49df-86df-82c2aa8077f2|spark_catalog.silver.silver_ecg|NULL       |s3a://warehouse/silv

In [11]:
# from pyspark.sql.functions import current_timestamp, col

# # Now read Avro
# avro_path = "s3a://warehouse/fs/ecg_avro/D_01/*.avro"
# df = spark.read.format("avro").load(avro_path)

# # ---------- FLATTEN ----------
# df_flat = df.select(
#     "device_id",
#     "record_id",
#     "start_time",
#     "ts",
#     col("signals.ecg_i"),
#     col("signals.ecg_i_filtered"),
#     current_timestamp().alias("ingest_ts")  # add ingest timestamp
# )

# # ---------- WRITE TO DELTA TABLE ----------
# df_flat.write.format("delta").mode("append").saveAsTable("silver.silver_ecg")

25/12/28 22:09:16 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://warehouse/fs/ecg_avro/D_01/*.avro.
java.io.FileNotFoundException: No such file or directory: s3a://warehouse/fs/ecg_avro/D_01/*.avro
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:4156)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:4007)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$getFileStatus$22(S3AFileSystem.java:3984)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.invokeTrackingDuration(IOStatisticsBinding.java:547)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.lambda$trackDurationOfOperation$5(IOStatisticsBinding.java:528)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.trackDuration(IOStatisticsBinding.java:449)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.trackDurationAndSpan(S3AFileSystem.java:2865)
	at org.apache.hadoop.fs.s3a.S3AFileSyst